In [63]:
import numpy as np
import pandas as pd
import re
import nltk
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
from nltk.corpus import stopwords
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import random
import pprint

In [64]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Read in data

In [65]:
peek = 10
def present_list_like(name, list_like, peek=peek):
    print(f"{name} peek:")
    print('  ' + '\n  '.join([str(seq) for seq in list_like[0:peek]]))

In [66]:
#Read data from covid_lies.csv dataset into dataframe
df = pd.read_csv('./data/covid_lies.csv')
print("The dataset:")
df.info()
print("\nData peek:")
print(df.head(peek))
print()

#Seperate out text data and labels
input_text = df['misconception'].to_numpy()
input_label = df['label'].to_numpy()
print("Unique labels:", np.unique(input_label))
orig_label_counts = df['label'].value_counts(normalize=True)
print("Counts of 'labels':", orig_label_counts)

The dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6591 entries, 0 to 6590
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   misconception_id  6591 non-null   int64 
 1   misconception     6591 non-null   object
 2   tweet_id          6591 non-null   int64 
 3   label             6591 non-null   object
dtypes: int64(2), object(2)
memory usage: 206.1+ KB

Data peek:
   misconception_id                                      misconception  \
0                 3             Coronavirus is genetically engineered.   
1                30  Blowing conch shells destroys coronavirus pote...   
2                57  Swans and dolphins swimming in Venice canals f...   
3                22                         Cocaine cures coronavirus.   
4                32  Observing janata curfew will result in the red...   
5                25  Holy communion cannot be the cause of the spre...   
6                61  Li

## Preprocess input data

In [68]:
def preprocess_text(text)->str:
    #Letter-level cleaning
    text = text.lower()
    valid_asciis = {9, *range(32, 127)}
    text = ''.join(filter(lambda x: ord(x) in valid_asciis, text))

    #Word/sequence-level cleaning
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'http\S+', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text


In [69]:
#Preprocess the text
for i in range(len(input_text)):
    input_text[i] = preprocess_text(input_text[i])

## Tokenize input text data

In [70]:
#Tokenize the text

input_tokens = [nltk.word_tokenize(text) for text in input_text]

# Split tokens based on non-alphanumeric characters
final_tokens = []
total_tokens = 0
for token_set in input_tokens:
    final_tkn_set = []
    for tk in token_set:
        sub_tkns = [c for c in re.split(r"(\W+)", tk) if c]
        total_tokens += len(sub_tkns)
        final_tkn_set += sub_tkns
    final_tokens.append(final_tkn_set)
present_list_like(f"Tokenized sentences({len(final_tokens)} sentences, {total_tokens} tokens)", final_tokens)

Tokenized sentences(18447 sentences, 133252 tokens) peek:
  ['coronavirus', 'genetically', 'engineered', '.']
  ['blowing', 'conch', 'shells', 'destroys', 'coronavirus', 'potency', '.']
  ['swans', 'dolphins', 'swimming', 'venice', 'canals', 'following', 'covid', '-', '19', 'lockdown', '.']
  ['cocaine', 'cures', 'coronavirus', '.']
  ['observing', 'janata', 'curfew', 'result', 'reduction', 'covid', '-', '19', 'cases', '40', '%', '.']
  ['holy', 'communion', 'can', 'not', 'cause', 'spread', 'coronavirus']
  ['lions', 'freed', 'keep', 'people', 'streets', 'moscow', '.']
  ['coronavirus', 'genetically', 'engineered', '.']
  ['cannabis', 'protects', 'covid', '-', '19', '.']
  ['safe', 'individuals', 'infected', 'covid', '-', '19', 'go', 'work', '.']


## Form embeddings for input data

In [71]:
#Embed the tokens

# Map each token to its frequency in the dataset
flat_tokens = [word for token_set in final_tokens for word in token_set]
frequencies = Counter(flat_tokens)
token_to_idx = {word: idx+1 for idx, (word, _) in enumerate(frequencies.most_common())}
vocab_size = len(token_to_idx)
print(vocab_size, "unique tokens")
present_list_like("Unique tokens", list(token_to_idx.keys()))

# Embed the tokens
freq_indexed = [[token_to_idx[token] for token in token_set] for token_set in final_tokens]

# Make embeddings the same size
forced_idx_set_size = max(len(idxs) for idxs in freq_indexed)
freq_indexed = [
    idxs[:forced_idx_set_size] + [0]*(forced_idx_set_size - len(idxs))
    for idxs in freq_indexed
]
present_list_like(f"\nFinal Index Sets(Set_Size = {forced_idx_set_size}, {len(freq_indexed)} index sets)", freq_indexed)

266 unique tokens
Unique tokens peek:
  .
  -
  coronavirus
  covid
  19
  deadly
  seasonal
  flu
  survive
  cure

Final Index Sets(Set_Size = 19, 18447 index sets) peek:
  [3, 15, 16, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [134, 135, 136, 137, 3, 138, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [165, 166, 167, 99, 100, 62, 4, 2, 5, 63, 1, 0, 0, 0, 0, 0, 0, 0, 0]
  [168, 30, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [241, 242, 243, 244, 245, 4, 2, 5, 246, 247, 248, 1, 0, 0, 0, 0, 0, 0, 0]
  [22, 23, 17, 18, 24, 19, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [169, 170, 171, 172, 96, 173, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [3, 15, 16, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [174, 66, 4, 2, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [108, 109, 74, 4, 2, 5, 110, 111, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Define and train a GRU model using PyTorch

## Save Model Weights

## Evaluate Model